# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# read in the .csv
ct_data = "..\output_data\cities.csv"
city_df = pd.read_csv(ct_data)
del city_df['Unnamed: 0']
city_df[['Latitude' , 'Longitude']] = city_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
city_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
0,Donegal,54.6500,-8.1167,68.04,63,90,3.56,IE,09-12-2021
1,Ushuaia,-54.8000,-68.3000,42.46,70,75,0.00,AR,09-12-2021
2,Bairiki Village,1.3292,172.9752,82.06,74,97,21.88,KI,09-12-2021
3,George Town,5.4112,100.3354,80.89,82,20,0.00,MY,09-12-2021
4,Adrar,20.5022,-10.0711,104.59,15,89,10.74,MR,09-12-2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
import gmaps

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# Create the heatmaps (used example form anthony)
fig = gmaps.figure(center=[0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# creating a new dataframe for only those cities that are between 70 and 80 degrees, windspeed less than 10
ideal_city = city_df.loc[(city_df['Tempature']>=70) & (city_df['Tempature'] <=80)]
ideal_city = ideal_city.loc[(ideal_city['Windspeed'] <= 10)]
ideal_city = ideal_city.loc[(ideal_city['Cloudiness'] <= 0)]
ideal_city.dropna()
ideal_city.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
130,Kitob,39.0843,66.8332,72.18,19,0,5.39,UZ,09-12-2021
179,Rosetta,31.4044,30.4164,78.80,64,0,6.08,EG,09-12-2021
258,Faro,37.0194,-7.9322,76.78,57,0,8.05,PT,09-12-2021
298,Qingdao,36.0986,120.3719,73.00,88,0,4.47,CN,09-12-2021
324,Inhambane,-23.8650,35.3833,75.18,60,0,9.22,MZ,09-12-2021


In [16]:
# stor into new hotel_df
hotel_df = ideal_city.reset_index(drop=True)
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather,Hotel Name
0,Kitob,39.0843,66.8332,72.18,19,0,5.39,UZ,09-12-2021,
1,Rosetta,31.4044,30.4164,78.80,64,0,6.08,EG,09-12-2021,
2,Faro,37.0194,-7.9322,76.78,57,0,8.05,PT,09-12-2021,
3,Qingdao,36.0986,120.3719,73.00,88,0,4.47,CN,09-12-2021,
4,Inhambane,-23.8650,35.3833,75.18,60,0,9.22,MZ,09-12-2021,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {
    "type" : "hotel",
    "keyword": "hotel",
    "radius": "5000",    
    "key": g_key
}



In [18]:
# Create a loop through each city and lat/lon in the datafraem
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row["Longitude"]
    city_name = row["City"]

    params["location"] = f"{lat},{lng}"

     # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Kitob.
Closest hotel in Kitob is GISAR ASIA HOTEL.
------------
Retrieving Results for Index 1: Rosetta.
Closest hotel in Rosetta is فندق النيل برشيد.
------------
Retrieving Results for Index 2: Faro.
Closest hotel in Faro is ibis Faro Algarve.
------------
Retrieving Results for Index 3: Qingdao.
Closest hotel in Qingdao is Shangri-La Hotel Qingdao.
------------
Retrieving Results for Index 4: Inhambane.
Closest hotel in Inhambane is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 5: San Quintín.
Closest hotel in San Quintín is Don Eddie's Landing.
------------
Retrieving Results for Index 6: Seoul.
Closest hotel in Seoul is Four Seasons Hotel Seoul.
------------
Retrieving Results for Index 7: Salmās.
Closest hotel in Salmās is Shahin hotel.
------------
Retrieving Results for Index 8: Zhangaözen.
Closest hotel in Zhangaözen is Hotel BasTau.
------------
Retrieving Results for Index 9: Baharly.
Missing field/result... skipping.
------

In [19]:
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(25)


,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather,Hotel Name
0,Kitob,39.0843,66.8332,72.18,19,0,5.39,UZ,09-12-2021,GISAR ASIA HOTEL
1,Rosetta,31.4044,30.4164,78.80,64,0,6.08,EG,09-12-2021,فندق النيل برشيد
2,Faro,37.0194,-7.9322,76.78,57,0,8.05,PT,09-12-2021,ibis Faro Algarve
3,Qingdao,36.0986,120.3719,73.00,88,0,4.47,CN,09-12-2021,Shangri-La Hotel Qingdao
4,Inhambane,-23.8650,35.3833,75.18,60,0,9.22,MZ,09-12-2021,Hotel Casa Do Capitão
5,San Quintín,30.4833,-115.9500,76.75,71,0,6.29,MX,09-12-2021,Don Eddie's Landing
6,Seoul,37.5683,126.9778,71.04,88,0,3.44,KR,09-12-2021,Four Seasons Hotel Seoul
7,Salmās,38.1973,44.7653,76.19,26,0,3.40,IR,09-12-2021,Shahin hotel
8,Zhangaözen,43.3000,52.8000,77.61,19,0,9.33,KZ,09-12-2021,Hotel BasTau
10,Nkhata Bay,-11.6066,34.2907,73.69,64,0,7.00,MW,09-12-2021,Mayoka Village


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude","Longitude"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))